## 1. Vorbereitung

In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
from ipywidgets import interactive
import matplotlib as mpl

%matplotlib widget
# %matplotlib notebook

### 1.1 Daten generieren

Generiere Trainings- und Testdaten mit einem Merkmal und einer kontinuierlichen Zielvariablen.
Zwischen den Variabeln herrscht ein logistischer Zusammenhang.

In [2]:
# der wahre funktionale Zusammenhang zwischen
def true_function(x):
    f = 5 / (1 + np.exp(-x + 2))
    return f

rng = np.random.RandomState(1)

# Daten - Beobachtungen / Merkmale
x_train = 10 * rng.rand(20)
x_test = 10 * rng.rand(20)

# Daten - Zielvariablen
y_train = true_function(x_train) + 0.5 * rng.randn(20)
y_test = true_function(x_test) + 0.5 * rng.randn(20)

# Zum Plotten
xx = np.linspace(0, 10)
ff = true_function(xx)

In [3]:
plt.figure()
# plt.plot(xx, ff)
plt.scatter(x_train, y_train, alpha=0.7)
plt.scatter(x_test, y_test, alpha=0.7)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 1.2 Vorbereitung der Plots

In [4]:
# convenience functions and variables zur Darstellung der Verlustfunktion
def empirical_risk(w, b, x_sample, y_sample):
    # makes heavy use of broadcasting
    W = np.repeat(w[..., np.newaxis], x_sample.shape[0], axis=-1)
    B = np.repeat(b[..., np.newaxis], x_sample.shape[0], axis=-1)
    Y_pred = W * x_sample + B
    loss = np.mean((Y_pred - y_sample)**2, axis=-1)
    return loss

def weight_norm(W, B):
    return W**2 + B**2

def L1_norm(W, B):
    return np.abs(W) + np.abs(B)

ws = np.linspace(-3, 3, 1000)
bs = np.linspace(-10, 10, 1000)

W, B = np.meshgrid(ws, bs)
L = empirical_risk(W, B, x_train, y_train)
L_reg = weight_norm(W, B)
L_reg_l1 = L1_norm(W, B)


L_test = empirical_risk(W, B, x_test, y_test)

L_min, L_max = L.min(), L.max()

## 2. Verlustfunktion optimieren

### 2.1. Random Search - 2D

Erzeugt ein interaktives Widget, in dem per Klick im rechten Subplot verschiedene Modelle instanziiert werden können und deren Verlust berechnet wird.

In [15]:
fig = plt.figure(figsize=(10, 4))

ax1 = fig.add_subplot(1, 2, 1)
ax2 = fig.add_subplot(1, 2, 2)

cax = fig.add_axes([0.95, 0.1, 0.02, 0.7])

ax1.set_ylim(-1, 8)
ax1.set_xlabel(r"$x$")
ax1.set_ylabel(r"$y$")

ax2.set_xlim(-3, 3)
ax2.set_ylim(-5, 5)
ax2.set_xlabel(r"$w$")
ax2.set_ylabel(r"$b$")
ax2.set_aspect("auto")

w = 0.1
b = 0.0

xx = np.linspace(0, 10, 100)
yy_hat = w * xx + b
y_hat_train = w * x_train + b

line_handle, = ax1.plot(xx, yy_hat, color="orange")
scatter_handle = ax1.scatter(x_train, y_train)
vline_handles = ax1.vlines(x_train.T, ymin=y_train.T, ymax=y_hat_train,
                           linestyle="dashed", color='r', alpha=0.3)

loss_points_handle = ax2.scatter([], [], s=10, alpha=0.5, cmap="plasma_r", vmin=L_min, vmax=L_max)
# im = ax2.scatter(W.flatten(), B.flatten(), s=10, alpha=0.5, cmap="plasma_r", vmin=L_min, vmax=L_max)
# im = ax2.imshow(L, cmap="plasma_r", vmin=L_min, vmax=L_max, aspe)
# text = ax2.text(0,0, "test", va="bottom", ha="left")

fig.colorbar(
    mpl.cm.ScalarMappable(
        norm=mpl.colors.Normalize(vmin=L_min, vmax=L_max),
        cmap="plasma_r"
    ),
    cax=cax)

loss_points = []
losses = []
scaled_losses = []

def onclick(event):
    
    tx = 'button=%d, xdata=%f, ydata=%f' % (event.button, event.xdata, event.ydata)
    
    if event.inaxes == ax2:
        w = event.xdata
        b = event.ydata

        # global loss_points
        global loss_points_handle
        loss_points.append([w, b])
        loss = empirical_risk(w, b, x_train, y_train)
        scaled_loss = np.clip((loss - L_min) / (L_max - L_min), a_min=0.0, a_max=1.0)
        losses.append(loss)
        scaled_losses.append(scaled_loss)

        loss_points_handle.remove()
        loss_points_handle = ax2.scatter(
            *np.array(loss_points).T,
            c=losses,
            vmin=L_min,
            vmax=L_max,
            s=10, alpha=0.5, cmap="plasma_r"
        )

        # loss_points_handle.set_offsets(loss_points)
        # loss_points_handle.set_facecolors(loss_points_handle.get_cmap()(scaled_losses))
        # loss_points_handle.set_facecolors(scaled_losses)

        yy_hat = w * xx + b
        y_hat_train = w * x_train + b
        line_handle.set_data(xx, yy_hat)
        global vline_handles
        vline_handles.remove()
        vline_handles = ax1.vlines(x_train.T, ymin=y_train.T, ymax=y_hat_train,
                                   linestyle="dashed", color='r',alpha=0.3)


cid = fig.canvas.mpl_connect('button_press_event', onclick)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 2.2. Random Search V2 -  2D

Erzeugt ein interaktives Widget mit verbesserter Zufallssuche.

In [5]:
fig = plt.figure(figsize=(10, 4))

widths = [3, 3, 1]
ax1 = fig.add_subplot(1, 2, 1)
ax2 = fig.add_subplot(1, 2, 2)

cax = fig.add_axes([0.95, 0.1, 0.02, 0.7])

ax1.set_ylim(-1, 8)
ax1.set_xlabel(r"$x$")
ax1.set_ylabel(r"$y$")

ax2.set_xlim(-3, 3)
ax2.set_ylim(-5, 5)
ax2.set_xlabel(r"$w$")
ax2.set_ylabel(r"$b$")
ax2.set_aspect("auto")

w = 0.1
b = 0.0

xx = np.linspace(0, 10, 100)
yy_hat = w * xx + b
y_hat_train = w * x_train + b

line_handle, = ax1.plot(xx, yy_hat, color="orange")
scatter_handle = ax1.scatter(x_train, y_train)
vline_handles = ax1.vlines(x_train.T, ymin=y_train.T, ymax=y_hat_train,
                           linestyle="dashed", color='r', alpha=0.3)

red = np.array([[1., 0., 0., 1.]])
green = np.array([[0., 0.50196078, 0., 1.]])
points_handle = ax2.scatter([], [], s=10, alpha=0.5, cmap="plasma_r", vmin=L_min, vmax=L_max)
accepted_handle = ax2.scatter([], [], s=20, alpha=1.0,
                              linewidth=2, facecolors='none', edgecolors=green)
rejected_handle = ax2.scatter([], [], s=20, alpha=1.0,
                              linewidth=2, facecolors='none', edgecolors=red)

t1 = ax2.text(1, 4.0, "Best loss: ")
t2 = ax2.text(1, 3.5, r"Best $w$: ")
t3 = ax2.text(1, 3.0, r"Best $b$: ")

fig.colorbar(
    mpl.cm.ScalarMappable(
        norm=mpl.colors.Normalize(vmin=L_min, vmax=L_max),
        cmap="plasma_r"
    ),
    cax=cax)

init = False
active_w = None
active_b = None

best_loss = np.inf
points = []
losses = []
accepted_points = []
rejected_points = []

sd = 0.5

def onclick(event):
    if event.inaxes == ax2:
        
        global init
        global best_loss
        global active_w
        global active_b
        global sd
        
        if not init:
            active_w = event.xdata
            active_b = event.ydata
            accepted_points.append([active_w, active_b])
            points.append([active_w, active_b])
            new_loss = empirical_risk(active_w, active_b, x_train, y_train)
            best_loss = new_loss
            losses.append(new_loss)
            init = True
        else:
            w = active_w + sd * np.random.randn()
            b = active_b + sd * np.random.randn()
            new_loss = empirical_risk(w, b, x_train, y_train)
            if new_loss < best_loss:
                active_w = w
                active_b = b
                accepted_points.append([active_w, active_b])
                best_loss = new_loss
            else:
                rejected_points.append([w, b])
            losses.append(new_loss)
            points.append([w, b])

        global points_handle
        global accepted_handle
        global rejected_handle
        
        t1.set_text(f"Best loss: {best_loss:.2f}")
        t2.set_text(fr"Best $w$: {active_w:.2f}")
        t3.set_text(fr"Best $b$: {active_b:.2f}")
        
        points_handle.remove()
        points_handle = ax2.scatter(
            *np.array(points).T,
            c=losses,
            vmin=L_min,
            vmax=L_max,
            s=10, alpha=0.5, cmap="plasma_r"
        )
        
        # decay out the color alpha value for the circles
        T_acc = len(accepted_points)
        T_rej = len(rejected_points)
        accepted_decay = np.exp(0.5*(np.arange(T_acc) - T_acc + 1))
        rejected_decay = np.exp(0.5*(np.arange(T_rej) - T_rej + 1))
        greens = np.repeat(green, T_acc, axis=0)
        greens[:, -1] *= accepted_decay
        reds = np.repeat(red, T_rej, axis=0)
        reds[:, -1] *= rejected_decay
        
        accepted_handle.remove()
        accepted_handle = ax2.scatter(
            *np.array(accepted_points).T,
            facecolors='none', edgecolors=greens, s=20
        )
        if len(rejected_points):
            try:
                rejected_handle.remove()
            except ValueError:
                pass
        
            rejected_handle = ax2.scatter(
                *np.array(rejected_points).T,
                facecolors='none', edgecolors=reds, s=20
            )
        
        # plot the current fit        
        yy_hat = active_w * xx + active_b
        y_hat_train = active_w * x_train + active_b
        line_handle.set_data(xx, yy_hat)
        global vline_handles
        vline_handles.remove()
        vline_handles = ax1.vlines(x_train.T, ymin=y_train.T, ymax=y_hat_train,
                                   linestyle="dashed", color='r',alpha=0.3)
        
        sd *= 0.995
    

cid = fig.canvas.mpl_connect('button_press_event', onclick)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 2.3 Random Search - 1D

Äquivalent zu 2.1. allerdings für eine Regression ohne Bias/Intercept, deshalb mit einer Verlustfunktion von nur einer Variablen (das Gewicht).

In [17]:
b = np.float64(1.22)

ws = np.linspace(-3, 3, 1000)
bs = b * np.ones_like(ws)

loss_function = empirical_risk(ws, bs, x_train, y_train)

plt.close("all")

fig = plt.figure(figsize=(10, 4))

widths = [3, 3, 1]
# gs = fig.add_gridspec(ncols=3, nrows=1, width_ratios=widths)
ax1 = fig.add_subplot(1, 2, 1) # (gs[0, 0])
ax2 = fig.add_subplot(1, 2, 2) # (gs[0, 1])

ax1.set_ylim(-1, 8)
ax1.set_xlabel(r"$x$")
ax1.set_ylabel(r"$y$")

ax2.set_xlim(-3, 3)
ax2.set_ylim(L_min - 10, 300)
ax2.set_xlabel(r"$w$")
ax2.set_ylabel(r"$L$")
ax2.set_aspect("auto")

w = 0.1

xx = np.linspace(0, 10, 100)
yy_hat = w * xx + b
y_hat_train = w * x_train + b

line_handle, = ax1.plot(xx, yy_hat, color="orange")
scatter_handle = ax1.scatter(x_train, y_train)
vline_handles = ax1.vlines(x_train.T, ymin=y_train.T, ymax=y_hat_train,
                           linestyle="dashed", color='r', alpha=0.3)

red = np.array([[1., 0., 0., 1.]])
green = np.array([[0., 0.50196078, 0., 1.]])
points_handle = ax2.scatter([], [], s=10, alpha=0.5)
accepted_handle = ax2.scatter([], [], s=20, alpha=1.0,
                              linewidth=2, facecolors='none', edgecolors=green)
rejected_handle = ax2.scatter([], [], s=20, alpha=1.0,
                              linewidth=2, facecolors='none', edgecolors=red)

t1 = ax2.text(1, 250, "Best loss: ")
t2 = ax2.text(1, 225, r"Best $w$: ")
t3 = ax2.text(1, 200, fr"Fixed $b$: {b}")


init = False

points = []
losses = []

def onclick(event):
    if event.inaxes == ax2:
        
        global init
        global w

        w = event.xdata
        points.append([w, b])
        current_loss = empirical_risk(w, b, x_train, y_train)
        losses.append(current_loss)

        global points_handle
        
        t1.set_text(f"Current loss: {current_loss:.2f}")
        t2.set_text(fr"Current $w$: {w:.2f}")
        t3.set_text(fr"Fixed $b$: {b:.2f}")
        
        points_handle.remove()
        points_handle = ax2.scatter(
            np.array(points)[:, 0],
            losses,
            c="blue",
            s=10, alpha=0.5
        )
        
        # plot the current fit        
        yy_hat = w * xx + active_b
        y_hat_train = w * x_train + active_b
        line_handle.set_data(xx, yy_hat)
        global vline_handles
        vline_handles.remove()
        vline_handles = ax1.vlines(x_train.T, ymin=y_train.T, ymax=y_hat_train,
                                   linestyle="dashed", color='r',alpha=0.3)


cid = fig.canvas.mpl_connect('button_press_event', onclick)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 2.4. Random Search V2 - 1D

Äquivalent zu 2.2. allerdings für eine Regression ohne Bias/Intercept, deshalb mit einer Verlustfunktion von nur einer Variablen (das Gewicht).

In [18]:
b = np.float64(1.22)

ws = np.linspace(-3, 3, 1000)
bs = b * np.ones_like(ws)

loss_function = empirical_risk(ws, bs, x_train, y_train)

plt.close("all")


fig = plt.figure(figsize=(10, 4))

ax1 = fig.add_subplot(1, 2, 1)
ax2 = fig.add_subplot(1, 2, 2)

ax1.set_ylim(-1, 8)
ax1.set_xlabel(r"$x$")
ax1.set_ylabel(r"$y$")

ax2.set_xlim(-3, 3)
ax2.set_ylim(-10, 300)
ax2.set_xlabel(r"$w$")
ax2.set_ylabel(r"$L$")
ax2.set_aspect("auto")

w = 0.1

xx = np.linspace(0, 10, 100)
yy_hat = w * xx + b
y_hat_train = w * x_train + b

line_handle, = ax1.plot(xx, yy_hat, color="orange")
scatter_handle = ax1.scatter(x_train, y_train)
vline_handles = ax1.vlines(x_train.T, ymin=y_train.T, ymax=y_hat_train,
                           linestyle="dashed", color='r', alpha=0.3)

red = np.array([[1., 0., 0., 1.]])
green = np.array([[0., 0.50196078, 0., 1.]])
points_handle = ax2.scatter([], [], s=10, alpha=0.5)
accepted_handle = ax2.scatter([], [], s=20, alpha=1.0,
                              linewidth=2, facecolors='none', edgecolors=green)
rejected_handle = ax2.scatter([], [], s=20, alpha=1.0,
                              linewidth=2, facecolors='none', edgecolors=red)

t1 = ax2.text(1, 250, "Best loss: ")
t2 = ax2.text(1, 225, r"Best $w$: ")
t3 = ax2.text(1, 200, fr"Fixed $b$: {b}")


init = False
active_w = None

best_loss = np.inf
points = []
losses = []
accepted_losses = []
accepted_points = []
rejected_losses = []
rejected_points = []

sd = 0.5

def onclick(event):
    if event.inaxes == ax2:
        
        global init
        global best_loss
        global active_w
        global sd
        
        if not init:
            active_w = event.xdata
            accepted_points.append([active_w, b])
            points.append([active_w, b])
            new_loss = empirical_risk(active_w, b, x_train, y_train)
            best_loss = new_loss
            losses.append(new_loss)
            accepted_losses.append(new_loss)
            init = True
        else:
            # w = event.xdata
            # b = event.ydata
            w = active_w + sd * np.random.randn()
            new_loss = empirical_risk(w, b, x_train, y_train)
            if new_loss < best_loss:
                active_w = w
                accepted_points.append([active_w, b])
                accepted_losses.append(new_loss)
                best_loss = new_loss
            else:
                rejected_points.append([w, b])
                rejected_losses.append(new_loss)
            losses.append(new_loss)
            points.append([w, b])

        global points_handle
        global accepted_handle
        global rejected_handle
        
        t1.set_text(f"Best loss: {best_loss:.2f}")
        t2.set_text(fr"Best $w$: {active_w:.2f}")
        t3.set_text(fr"Fixed $b$: {b:.2f}")
        
        points_handle.remove()
        points_handle = ax2.scatter(
            np.array(points)[:, 0],
            losses,
            c="blue",
            s=10, alpha=0.5
        )
        # change offsets to encircle accepted/rejected points
        # accepted_handle.set_offsets(accepted_points)
        # rejected_handle.set_offsets(rejected_points)
        
        # decay out the color alpha value for the circles
        T_acc = len(accepted_points)
        T_rej = len(rejected_points)
        accepted_decay = np.exp(0.5*(np.arange(T_acc) - T_acc + 1))
        rejected_decay = np.exp(0.5*(np.arange(T_rej) - T_rej + 1))
        greens = np.repeat(green, T_acc, axis=0)
        greens[:, -1] *= accepted_decay
        reds = np.repeat(red, T_rej, axis=0)
        reds[:, -1] *= rejected_decay
        
        accepted_handle.remove()
        accepted_handle = ax2.scatter(
            np.array(accepted_points)[:, 0],
            accepted_losses,
            facecolors='none', edgecolors=greens, s=25
        )
        if len(rejected_points):
            try:
                rejected_handle.remove()
            except ValueError:
                pass
        
            rejected_handle = ax2.scatter(
                np.array(rejected_points)[:, 0],
                rejected_losses,
                facecolors='none', edgecolors=reds, s=20
            )
        
        # plot the current fit        
        yy_hat = active_w * xx + b
        y_hat_train = active_w * x_train + b
        line_handle.set_data(xx, yy_hat)
        global vline_handles
        vline_handles.remove()
        vline_handles = ax1.vlines(x_train.T, ymin=y_train.T, ymax=y_hat_train,
                                   linestyle="dashed", color='r',alpha=0.3)
        sd *= 0.995

cid = fig.canvas.mpl_connect('button_press_event', onclick)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 2.5 Gradientenabstieg

Illustration des Gradientenabstiegsverfahrens per interaktivem Widget.

In [19]:
b = np.float64(1.22)

def gradient_w(w, b, x_sample, y_sample):
    y_hat = w * x_sample + b
    error = y_hat - y_sample
    grad_w = 2/x_sample.shape[0] * np.sum(error * x_sample)  # works only in 1D
    return grad_w

def tangent(w_sample, w, grad_w, loss_w):
    tangent_b = loss_w - w * grad_w
    tangent_sample = grad_w * w_sample + tangent_b
    return tangent_sample

ws = np.linspace(-3, 3, 1000)
bs = b * np.ones_like(ws)

loss_function = empirical_risk(ws, bs, x_train, y_train)

In [28]:
plt.close("all")

fig = plt.figure(figsize=(10, 4))

widths = [3, 3, 1]
# gs = fig.add_gridspec(ncols=3, nrows=1, width_ratios=widths)
ax1 = fig.add_subplot(1, 2, 1) # (gs[0, 0])
ax2 = fig.add_subplot(1, 2, 2) # (gs[0, 1])

ax1.set_ylim(-1, 8)
ax1.set_xlabel(r"$x$")
ax1.set_ylabel(r"$y$")

ax2.set_xlim(-3, 3)
ax2.set_ylim(-10, 300)
ax2.set_xlabel(r"$w$")
ax2.set_ylabel(r"$L$")
ax2.set_aspect("auto")

w = 0.1
xx = np.linspace(0, 10, 100)
yy_hat = w * xx + b
y_hat_train = w * x_train + b

line_handle, = ax1.plot(xx, yy_hat, color="orange")
scatter_handle = ax1.scatter(x_train, y_train)
vline_handles = ax1.vlines(x_train.T, ymin=y_train.T, ymax=y_hat_train,
                           linestyle="dashed", color='r', alpha=0.3)

red = np.array([[1., 0., 0., 1.]])
green = np.array([[0., 0.50196078, 0., 1.]])
points_handle = ax2.scatter([], [], s=10, alpha=0.7)
tangent_handle, = ax2.plot([], [], alpha=0.3)
step_handle = ax2.arrow(-10, 0, -10, 0)
ax2.plot(ws, loss_function, color="cyan", alpha=0.3)

t1 = ax2.text(1, 250, "Current loss: ")
t2 = ax2.text(1, 225, r"Current $w$: ")
t3 = ax2.text(1, 200, fr"Fixed $b$: {b}")


init = False

points = []
losses = []

lambda_ = 0.1
grad_w = 0.0


def onclick(event):
    if event.inaxes == ax2:
        
        global init
        global best_loss
        global w
        global lambda_
        global grad_w
        
        if not init:
            w = event.xdata
            points.append([w, b])
            new_loss = empirical_risk(w, b, x_train, y_train)
            losses.append(new_loss)
            init = True
        else:
            w = w - lambda_ * grad_w
            new_loss = empirical_risk(w, b, x_train, y_train)
            losses.append(new_loss)
            points.append([w, b])

        global points_handle
        global tangent_handle
        global step_handle
        
        t1.set_text(f"Current loss: {new_loss:.2f}")
        t2.set_text(fr"Current $w$: {w:.2f}")
        t3.set_text(fr"Fixed $b$: {b:.2f}")
        
        points_handle.remove()
        points_handle = ax2.scatter(
            np.array(points)[:, 0],
            losses,
            c="blue",
            s=10, alpha=0.5
        )
        
        grad_w = gradient_w(w, b, x_train, y_train)
        tangent_handle.set_data(ws, tangent(ws, w, grad_w, new_loss))
        
        step_handle.remove()
        step_handle = ax2.arrow(w, new_loss, - lambda_ * grad_w, 0,
                                length_includes_head=True, shape="full",
                                width=1e-3, color="red")
        
        # plot the current fit        
        yy_hat = w * xx + b
        y_hat_train = w * x_train + b
        line_handle.set_data(xx, yy_hat)
        global vline_handles
        vline_handles.remove()
        vline_handles = ax1.vlines(x_train.T, ymin=y_train.T, ymax=y_hat_train,
                                   linestyle="dashed", color='r', alpha=0.3)
        lambda_ *= 0.99

cid = fig.canvas.mpl_connect('button_press_event', onclick)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …